## Notebook for making predictions with deleted constant y samples


In [126]:
import pandas as pd
import sys
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor

### Reading dataset stored in other notebooks

In [127]:
sys.path.append('../')

%store -r dml

dm = dml

## Getting data

In [128]:
X_A_train = dm.data_A.iloc[:, 2:-1]
Y_A_train = dm.data_A.iloc[:, 1]

X_B_train = dm.data_B.iloc[:, 2:-1]
Y_B_train = dm.data_B.iloc[:, 1]

X_C_train = dm.data_C.iloc[:, 2:-1]
Y_C_train = dm.data_C.iloc[:, 1]

X_A_test = dm.X_test_estimated_a[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]
X_B_test = dm.X_test_estimated_b[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]
X_C_test = dm.X_test_estimated_c[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]

Y_A_train


0         0.00
1         0.00
2         0.00
3         0.00
4        19.36
         ...  
21500     9.02
21501     0.00
21502     0.00
21503     0.00
21504     0.00
Name: pv_measurement, Length: 21483, dtype: float64

## Catboost models
will try with two models:
-   One for dataset A
-   One for B and C

In [129]:
catModel_a = CatBoostRegressor()
catModel_b = CatBoostRegressor()
catModel_c = CatBoostRegressor()

In [130]:
catModel_a.fit(X_A_train,Y_A_train)
catModel_b.fit(X_B_train,Y_B_train)
catModel_c.fit(X_C_train,Y_C_train)

Learning rate set to 0.066474
0:	learn: 1272.1128750	total: 8.22ms	remaining: 8.21s
1:	learn: 1210.0767868	total: 13.9ms	remaining: 6.93s
2:	learn: 1153.1612116	total: 20.7ms	remaining: 6.89s
3:	learn: 1101.4169163	total: 25.8ms	remaining: 6.43s
4:	learn: 1052.7347745	total: 31.1ms	remaining: 6.2s
5:	learn: 1008.1751562	total: 37.2ms	remaining: 6.16s
6:	learn: 968.3506599	total: 42.6ms	remaining: 6.05s
7:	learn: 930.9758289	total: 48.2ms	remaining: 5.98s
8:	learn: 895.9190184	total: 54.5ms	remaining: 6s
9:	learn: 864.8143806	total: 60.8ms	remaining: 6.02s
10:	learn: 835.8770111	total: 66.6ms	remaining: 5.99s
11:	learn: 810.1176363	total: 72.5ms	remaining: 5.97s
12:	learn: 785.6059348	total: 77.5ms	remaining: 5.89s
13:	learn: 763.4960752	total: 83.2ms	remaining: 5.86s
14:	learn: 742.9856892	total: 88.6ms	remaining: 5.82s
15:	learn: 724.7448337	total: 94.2ms	remaining: 5.79s
16:	learn: 709.2951665	total: 142ms	remaining: 8.19s
17:	learn: 694.5423954	total: 147ms	remaining: 8.02s
18:	lear

## Preds

In [131]:
pred_A = catModel_a.predict(X_A_test)
pred_B = catModel_b.predict(X_B_test)
pred_C = catModel_c.predict(X_C_test)

In [132]:
df_A = pd.DataFrame()

df_A["predict"] = pred_A
df_A["location"] = "A"

df_B = pd.DataFrame()

df_B["predict"] = pred_B
df_B["location"] = "B"

df_C = pd.DataFrame()

df_C["predict"] = pred_C
df_C["location"] = "C"

df_mid = pd.concat([df_A, df_B], ignore_index=True)

df = pd.concat([df_mid, df_C], join="inner", ignore_index=True)

df = df.drop("location", axis=1)

df[df<0] = 0

# NAME THE FILE 
df.to_csv("log_sub_1d.csv")

df.shape

(2160, 1)